# Aufgabe 3
Es sei eine Funktion gegeben, die die LR-Zerlegung einer nicht singul¨aren Matrix berechnet.

## a) Geben Sie einen Pseudocode fuer einen Algorithmus an, mit dem Sie mithilfe der LR- Zerlegung die Inverse einer Matrix ausrechnen koennen.

Warum Pseudocode und nicht Python? ist fast dasselbe.
- Die LR-Funktion ist gegeben
- Ich benutze noch eine Funktion, um LGS mit triagonalisierten Matrizen zu loesen, also einsetzen. Pseudocode fuer das Verfahren mit einer oberen Dreiecksmatrix (R) ist im Skript bei Algorithmus 3.2 schon vorhanden. Natuerlich ist der Code fuer L fast gleich.
  - Von daher benutze ich Funktionen aus der Bibliothek `scipy`, die diese Funktionalitaet schon implementieren
- **Die Idee ist**, fuer jede Spalte `Si` der Einheitsmatrix den Vektor `xi` zu finden, sodass `Axi = Si`. Dann sollte gelten `[x1...xn] = A^-1` 

In [17]:
import numpy as np
from scipy.linalg import lu as lr, solve_triangular

In [18]:
def solve_ls(L, R, b): # Gesamtzahl Operationen: 2 * n^2 
    z = solve_triangular(L, b, lower=True) # Anzahl Operationen: n^2 nach VL
    x = solve_triangular(R, z, lower=False) # Anzahl Operationen: n^2 nach VL
    return x


# A ist eine n x n Matrix
def get_inv(A): # Anzahl 'wesentlichen' Operationen: 1. + 3. = (2/3) * n^3 (1 + O(1/n)) + 2n^3 = 2n^3 (4/3 + O(1/n)/3)
    # LR-Zerlegung hat dieselbe Komplexitaet wie Gauss-Eliminiationsverfahren
    P,L,R = lr(A) # 1. Anzahl Operationen: (2/3) * n^3 (1 + O(1/n)) 
    
    # P besteht nur aus Transpositionen, womit deren Transponierte auch die Inverse ist
    # Um das einzusehen betrachte man z.B die Untergruppe von den Transpositionsmatrizen in der Gruppe der Elementarmatrizen
    # und benutze, dass jede Transposition die eigene inverse Matrix ist und dass (AB)^t = A^t B^t sowie (ab)^-1 = b^-1 a^-1 fuer 
    P_t = P.T # Anzahl Operationen: n*(n+1)/2 * O(1) mit Vertauschungsalgorithmus, aber wird wohl nicht zu den "wesentlichen" gezaehlt
    
    # wir verwenden die Spalten der Einheitsmatrix als b
    En = np.eye(A.shape[0]) # Anzahl Operationen: "Gratis"
    
    
    # fuer jede Spalte der Einheitsmatrix, hole den Vektor, mit dem ich A multiplizieren kann, sodass ich die Spalte bekomme
    # Ax = b -> PLRx = b -> LRx = P^-1 b -> LRx = P^t b, von daher P_t @ En[:, i]

    # P_t @ En[:, i] ist P^-1 * xi. Anzahl Operationen: O(2 * n^2) nach VL. Wird aber wieder wohl nicht zu den wesentlichen gezaehlt
    # 2. Fuer solve_ls (Also rueckwaerts und vorwaertseinsetzen) ist die Anzahl an Operationen: 2 * n^2 
    # 3. Beide werden n mal ausgefuehrt -> n * ((2 * n^2) + O(2 * n^2))= 2n^3 * (1  + O(1)) oder nur 2n^3
    all_x = [solve_ls(L, R, P_t @ En[:, i]) for i in range(En.shape[1])]


    # Fuege sie alle zu einer Matrix zusammen
    return np.column_stack(all_x)
    

    

In [19]:

# Beispiel aus https://www.cuemath.com/algebra/inverse-of-3x3-matrix/
A = np.array([
    [1,  2, -1],
    [2,  1,  2],
    [-1, 2,  1]
])


inv_A = get_inv(A)
print(inv_A)
print(A @ inv_A) 
# ^ Wegen Gleitarithmetik-Fehler sind nicht alle Eintraege genau 0, wo sie 0 sein sollten,
# aber man sieht, die sind sehr sehr klein (e-17)


[[ 0.1875  0.25   -0.3125]
 [ 0.25    0.      0.25  ]
 [-0.3125  0.25    0.1875]]
[[ 1.00000000e+00  0.00000000e+00 -2.77555756e-17]
 [ 0.00000000e+00  1.00000000e+00  5.55111512e-17]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## b) Geben Sie die Anzahl der wesentlichen Operationen des Algorithmus fuer eine n × n Matrix an. Sie koennen die Anzahl fuer die LR-Zerlegung, Vorwaerts- und Rueckwaertseinsetzen aus der Vorlesung verwenden.
Zu den Wesentlichen werden gezaehlt:
- Die LR-Zerlegung: `(2/3) * n^3 (1 + O(1/n))` nach VL
  - Passiert nur 1 mal
- Rueckwaerts- und Vorwaertseinsetzen: `n^2` nach VL
  - Passiert zweimal fuer jedes `xi`, also `2n` mal.
 
Insgesamt: `2n^3 (4/3 + O(1/n)/3)`. Siehe Kommentare im Code fuer mehr Details

In [22]:

# Zusatz fuer die Programmieraufgabe
B = np.array([[ 1,  3, -2],
              [-2, -5,  2],
              [ 3,  7, -7]])


inv_B = get_inv(B)
print(inv_B)
print(B @ inv_B) 
print("det B", np.linalg.det(B)) # != 0, B nicht singulaer

[[-4.2 -1.4  0.8]
 [ 1.6  0.2 -0.4]
 [-0.2 -0.4 -0.2]]
[[ 1.00000000e+00  0.00000000e+00  5.55111512e-17]
 [ 1.11022302e-16  1.00000000e+00 -5.55111512e-17]
 [ 2.49800181e-15  2.22044605e-16  1.00000000e+00]]
det B -5.000000000000001
